In [2]:
%run ./src/__init__.py

In [3]:
from time import process_time

In [4]:
df_artists = pd.read_csv('./data/artist_uris.csv', header=None, names=['name', 'uri'])

In [5]:
df_artists[['uri1','uri2','id']] = df_artists.uri.str.split(':',3,expand=True)
df_artists = df_artists.drop(columns=['uri1','uri2'])

In [6]:
df_artists

,name,uri,id
0,1:43,spotify:artist:39EHxSQAIaWusRqSI9xoyF,39EHxSQAIaWusRqSI9xoyF
1,2:00 AM,spotify:artist:4tN3rZ7cChj4Wns2Wt2Nj6,4tN3rZ7cChj4Wns2Wt2Nj6
2,2:15,spotify:artist:4HsOm6VNKZtGh8W8GhdNu4,4HsOm6VNKZtGh8W8GhdNu4
3,2:54,spotify:artist:3LsQKoRgMc8VEkQn66jfAQ,3LsQKoRgMc8VEkQn66jfAQ
4,4:20,spotify:artist:5KCG0FDMDPzQpxcohGUnyH,5KCG0FDMDPzQpxcohGUnyH
...,...,...,...
81318,黃曉明,spotify:artist:53F8atvCmVFVOvnKwZXBd3,53F8atvCmVFVOvnKwZXBd3
81319,黃玠瑋,spotify:artist:6VI0p0xTjBKKxQN8i8vGpD,6VI0p0xTjBKKxQN8i8vGpD
81320,黃義達,spotify:artist:7kaq0LysuRSgBZSorlZ7Vj,7kaq0LysuRSgBZSorlZ7Vj
81321,黃顯忠 (Huang Xianzhong),spotify:artist:7MuuEryyseo5cvReO6gdPF,7MuuEryyseo5cvReO6gdPF


In [7]:
t_start = process_time()

genres_list = set([])

for i in range(0,81350,50):
    ids = df_artists[i:i+49]['id']
    features = sp.artists(ids)['artists']
    for feature in features:
        genres_list = genres_list.union(set(feature['genres']))
    
t_end = process_time()
print('Duration:',t_end-t_start)

Duration: 48.521461306


In [8]:
len(genres_list)

4738

In [8]:
genres_list_1 = list(genres_list)
genres_list_1.sort()

all_genres = pd.DataFrame({'genre':genres_list_1})
all_genres

,genre
0,21st century classical
1,432hz
2,48g
3,5th wave emo
4,8-bit
...,...
4733,zouglou
4734,zouk
4735,zouk riddim
4736,zurich indie


In [98]:
seed_genres = sp.recommendation_genre_seeds()['genres']

print(seed_genres)

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

In [9]:
seed_genres = sp.recommendation_genre_seeds()['genres']

extra_words =['hip hop','trap','rap','r&b','afro','instrumental','choir','orchestra','banda','bachata','vallenato','balkan','brasileiro','romanticism','cubano',
              'psychedelic','death','perreo','infantil','musica.+','musique.+','progressive','swing','romantic','trova','christian','experimental','bolero',
             'brass band','soundtrack','celtic','drill','carnaval','cha-cha-cha','bop','cumbia','surf','vintage','vocal']
remove_words = ['british','german','french','swedish','spanish']

seed_genres = seed_genres + extra_words
seed_genres_clean = [gen for gen in seed_genres if gen not in remove_words]


str_seed = '('+'|'.join(seed_genres_clean)+')'

all_genres['seed_genre'] = all_genres['genre'].str.extract(str_seed)

print(len(all_genres[all_genres.seed_genre.isnull()==False]))

all_genres.to_csv('./data/seed_genres.csv',header=True, index=False)

3516


In [17]:
all_genres_clean = all_genres[all_genres.seed_genre.isnull()==False]
all_genres_clean = all_genres_clean.reset_index(drop=True)

extra_words_replace = ['hip-hop','hip-hop','hip-hop','r-n-b','afrobeat','instrumental','vocal','classical','latin','latin','latin','folk','latin','romantic','latin',
                       'rock','metal','reggaeton','kids','world-music','world-music','rock','jazz','romantic','latin','christian','experimental','latin','jazz',
                       'movies','folk','hip-hop','folk','latin','jazz','latin','ska','jazz','vocal']

all_genres_clean['seed_genre'] = all_genres_clean['seed_genre'].replace(extra_words, extra_words_replace, regex=True)

all_genres_clean.to_csv('./data/seed_genres_clean.csv',header=True, index=False)

In [11]:
all_genres_clean

,genre,seed_genre
0,21st century classical,classical
1,5th wave emo,emo
2,aberdeen indie,indie
3,abstract hip hop,hip-hop
4,abstract idm,idm
...,...,...
3511,zim gospel,gospel
3512,zim hip hop,hip-hop
3513,zim urban groove,groove
3514,zimdancehall,dance


In [12]:
len(all_genres_clean['seed_genre'].unique())

88

In [13]:
len(seed_genres)

165

In [18]:
all_genres_test = pd.read_csv('./data/seed_genres_clean.csv')
all_genres_test

,genre,seed_genre
0,21st century classical,classical
1,5th wave emo,emo
2,aberdeen indie,indie
3,abstract hip hop,hip-hop
4,abstract idm,idm
...,...,...
3511,zim gospel,gospel
3512,zim hip hop,hip-hop
3513,zim urban groove,groove
3514,zimdancehall,dance


In [19]:
GENRES_DICT = dict(zip(all_genres_clean.genre,all_genres_clean.seed_genre))
#GENRES_DICT

In [22]:
genres_list = list(set(GENRES_DICT.values()))
genres_list.sort()

for col in genres_list:
    df_artists[col] = 0

In [ ]:
t_start = process_time()

for i in range(0,1000,50):
    ids = df_artists[i:i+49]['id']
    features = sp.artists(ids)['artists']
    for my_id, feature in zip(ids,features):
        for gen in feature['genres']:
            try:
                df_artists.loc[df_artists['id']==my_id][gen] = 1
            except:
                pass
    
t_end = process_time()
print('Duration:',t_end-t_start)

<ipython-input-27-0a00e2faf7a9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_artists.loc[df_artists['id']==my_id][gen] = 1


In [ ]:
df_artists